In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\keb7\AppData\Local\Temp\ipykernel_32060\1140549349.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\stat.py

In [2]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
model = LinearRegression()

transform1 = make_column_transformer(
    (OneHotEncoder(), ["industry"]),
    remainder="passthrough"
)
transform2 = PolynomialFeatures(degree=2)
pipe = make_pipeline(
    transform1,
    transform2,
    model
)



server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

df = pd.read_sql(
    """
    select ticker, date, ret, roeq, bm, siccd
    from data
    order by ticker, date
    """, 
    conn
)

df = df.dropna()
df["actual"] = df.ret

qt = QuantileTransformer(output_distribution="normal")

grouped = df.groupby("date", group_keys=False)

features = ["roeq", "bm"]

df[features+["ret"]] = grouped[features+["ret"]].apply(
  lambda d: 
    pd.DataFrame(
      qt.fit_transform(d),
      columns=d.columns,
      index=d.index
    )     
)

inds = pd.read_csv(
  "files/siccodes12.csv", 
  index_col="industry"
)

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"

codes = pd.Series({code: industry(code) for code in df.siccd.unique()})
codes.name = "industry" 
codes.index.name = "siccd"
df = df.merge(codes, on="siccd")
features.append("industry")

df = df.set_index(["date", "ticker"])
dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
predictions = None
for train_date, end_date in zip(dates[:-1], dates[1:]):
  fltr1 = df.index.get_level_values("date") < train_date
  fltr2 = df.index.get_level_values("date") < end_date
  train = df[fltr1]
  test = df[~fltr1 & fltr2]
  Xtrain = train[features]
  ytrain = train["ret"]
  Xtest = test[features]
  pipe.fit(Xtrain, ytrain)
  pred = pipe.predict(Xtest)
  pred = pd.Series(pred, index=test.index)
  predictions = pd.concat((predictions, pred))
df["predict"] = predictions

numlong = 100
numshort = 100

df["rank_from_top"] = df.groupby("date").predict.rank(
  method="first", 
  ascending=False
)
df["long"] = df.rank_from_top <= numlong

df["rank_from_bottom"] = df.groupby("date").predict.rank(
  method="first"
)
df["short"] = df.rank_from_bottom <= numshort

df = df[df.index.get_level_values("date") >= dates[0]]

long_ret = df.groupby("date").apply(
    lambda d: (d.long*d.ret).mean()
)
short_ret = df.groupby("date").apply(
    lambda d: (d.short*d.ret).mean()
)
ret = 1.5*long_ret - 0.5*short_ret
ret.head()

date
2005-01    0.016546
2005-02    0.022419
2005-03    0.004282
2005-04    0.008603
2005-05    0.002557
dtype: float64